In [1]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
import math
from torch.autograd import Variable
import copy
import pandas
import Queue
import pyedflib
from os import walk

In [2]:
dir = "np_data13"
files = []
data_set_t = []
target_t = []

for (dirpath, dirnames, filenames) in walk(dir):
    files.extend(filenames)
    break
for instance, file  in enumerate(files):
    if (file == "T0.npy" or file == "T1.npy" or file == "T2.npy" or file == "T3.npy" or file == "T4.npy" or file == "T5.npy"):
        continue
    if (file == "0.npy" or file == "1.npy" or file == "2.npy" or file == "3.npy" or file == "4.npy" or file == "5.npy"):
        data_set_t.append(np.load(dir + '/' +file))
        target_t.append(np.load(dir + '/T' +file))
calm_test= np.load(dir + '/calm_test.npy')
calm_train = np.load(dir + '/calm_train.npy')

Tcalm_test= np.load(dir + '/Tcalm_test.npy')
Tcalm_train = np.load(dir + '/Tcalm_train.npy')


In [3]:

"""temp = data_set_t[0]
for i in range (5):
    temp = np.vstack(data_set_t[i+1])"""
data_set = np.vstack((data_set_t[0],data_set_t[1],data_set_t[2],data_set_t[3],data_set_t[4], data_set_t[5]))
targets = np.hstack((target_t[0],target_t[1],target_t[2],target_t[3],target_t[4],  target_t[5]))
targets.shape

(8628,)

In [105]:
8628 % 256

180

In [4]:
indexes = np.arange(data_set.shape[0])
np.random.shuffle(indexes)

data_set = data_set[indexes]
targets = targets[indexes]

"""tst_data = np.zeros((data_set.shape[0]%256, 62, 1200))
tst_targs = np.zeros((data_set.shape[0]%256))"""

tst_data = copy.deepcopy(data_set[data_set.shape[0] - (data_set.shape[0]%256) :])
tst_targs = copy.deepcopy(targets[targets.shape[0] - (targets.shape[0]%256) :])

val_data = copy.deepcopy(data_set[:256])
val_targs = copy.deepcopy(targets[:256])
print (tst_data.shape)

data_set = data_set[256:(data_set.shape[0] - (data_set.shape[0]%256))]
targets = targets[256:(targets.shape[0] - (targets.shape[0]%256))]

8448


In [ ]:
indexes = np.arange(data_set.shape[0])
np.random.shuffle(indexes)

data_set = data_set[indexes]
targets = targets[indexes]

"""tst_data = np.zeros((data_set.shape[0]%256, 62, 1200))
tst_targs = np.zeros((data_set.shape[0]%256))"""

tst_data = copy.deepcopy(data_set[data_set.shape[0] - (data_set.shape[0]%256) :, 150:250])
tst_targs = copy.deepcopy(targets[targets.shape[0] - (targets.shape[0]%256) :])

val_data = copy.deepcopy(data_set[:256, 150:250])
val_targs = copy.deepcopy(targets[:256])
print (data_set.shape[0] - (data_set.shape[0]%256))

data_set = data_set[256:(data_set.shape[0] - (data_set.shape[0]%256)), 150:250]
targets = targets[256:(targets.shape[0] - (targets.shape[0]%256))]

In [25]:
val_targs.shape

(256,)

In [5]:
tst_data = np.vstack((calm_test[:120,], tst_data))

tst_data.shape

tst_targs = np.hstack((Tcalm_train[:120], tst_targs))
print (tst_targs.shape, tst_data.shape)
val_data = np.vstack((calm_test[120:248], val_data))

val_targs = np.hstack((Tcalm_test[120:248], val_targs))
print ( val_targs.shape)
print (val_data.shape)

(300,) (300, 500, 31)
(384,)
(384, 500, 31)


In [6]:
val_data = np.transpose(val_data, (0, 2, 1))
tst_data = np.transpose(tst_data, (0, 2, 1))

In [7]:
data_set = np.vstack((data_set, calm_train))
targets = np.hstack((targets, Tcalm_train))

In [8]:
data_set = np.transpose(data_set, (0, 2, 1))

In [9]:
print (data_set.shape,targets.shape )
print (10424 % 256)

(11192, 31, 500) (11192,)
184


In [50]:
7424+256+239

7919

In [111]:
data_set.shape[0]/256

43

In [10]:
def get_data(dataset, targets, batch_size):
    indexes = np.arange(dataset.shape[0])
    np.random.shuffle(indexes)
    
    dataset = dataset[indexes]
    targets = targets[indexes]
    
    tr_data = np.zeros((dataset.shape[0]/batch_size, batch_size, 31, 500))   
    tr_targs = np.zeros((dataset.shape[0]/batch_size, batch_size))
    
    for i in range (dataset.shape[0]/batch_size):
            
        tr_data[i] = copy.deepcopy(dataset[i*batch_size:(i+1)*batch_size])
        tr_targs[i] = copy.deepcopy(targets[i*batch_size: (i+1)*batch_size])
        
    return tr_data, tr_targs

In [15]:
class LikeDislikeModel(nn.Module):
    def __init__(self, 
                 in_channels,
                 out_size,
                 L_sizes = [300, 200, 300],
                 net = None,
                 fc_sizes = [700, 250],
                 n_local_pred = 3,
                 f_sizes = [3, 3, 5], 
                 channels = [256, 128, 64], 
                 strides = [2, 2, 3], 
                 use_cuda = True):
        
        super(LikeDislikeModel, self).__init__()
        
        self.in_channels = in_channels

        self.fc_sizes = fc_sizes
        self.out_size = out_size
        self.n_local_pred = n_local_pred
        self.use_cuda = use_cuda
        self.L_sizes = L_sizes
        self.net = net
        
        self.f_sizes = f_sizes
        self.channels = channels
        self.strides = strides
        self.len_sizes = self._len_sizes()
        
        self.conv1 = nn.ModuleList()
        self.conv2 = nn.ModuleList()
        self.conv3 = nn.ModuleList()
        self.fc1 = nn.ModuleList()
        self.fc2 = nn.ModuleList()
        self.fc3 = nn.ModuleList()
        
        # TODO : add batchnormalization for each conv 
        self.batchnorm = nn.ModuleList()
        
        
        for i in range (n_local_pred):
            
            self.conv1.append(nn.Conv1d(in_channels, channels[0], f_sizes[0], strides[0]))
            self.conv2.append(nn.Conv1d(channels[0], channels[1], f_sizes[1], strides[1]))               
            self.conv3.append(nn.Conv1d(channels[1], channels[2], f_sizes[2], strides[2]))
            
            self.fc1.append(nn.Linear(self.len_sizes[i][-1] * self.channels[-1], fc_sizes[0]))
            self.fc2.append(nn.Linear(fc_sizes[0], fc_sizes[1]))
            self.fc3.append(nn.Linear(fc_sizes[1], out_size))
            self.batchnorm.append(nn.ModuleList())
            for convNum in range(3):
                self.batchnorm[i].append(nn.BatchNorm1d(channels[convNum]))
            
        self.dropout = nn.Dropout(p=0.5)
        
    def _len_sizes(self):
        len_sizes = []
        for i in range (self.n_local_pred):
            temp = []
            for j in range (len(self.f_sizes)):
                if j == 0:
                    temp.append(int(math.floor((self.L_sizes[i] - self.f_sizes[j])/self.strides[j] + 1)))
                else:
                    temp.append(int(math.floor((temp[j-1] - self.f_sizes[j])/self.strides[j] + 1)))
            len_sizes.append(copy.deepcopy(temp))
        return len_sizes

    def preprocessing(self, input_batch):
        nns = []
        if (self.n_local_pred == 1):
            if (self.net == 0):
                nns.append(input_batch[:, :, 200:500])
            elif (self.net == 1):
                nns.append(input_batch[:, :, 500:700])
            elif (self.net == 2):
                nns.append(input_batch[:, :, 700:1000])
            elif (self.net == 3):
                nns.append(input_batch)
            else:
                nns.append(input_batch[:, :, 200:1000])
        else:
            nns.append(input_batch[:, :, 200:500])
            nns.append(input_batch[:, :, 500:700])
            nns.append(input_batch[:, :, 700:1000])
        #print (nns[0].shape, len(nns))
        for i in range (self.n_local_pred):
            zero_mean = nns[i] - nns[i].mean()
            for n in range(input_batch.shape[0]):
                nns[i][n] = zero_mean[n, :, :]/(np.std(zero_mean[n, :, :],axis = 0))
                
            if self.use_cuda:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i]))).cuda()
            else:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i])))
        return nns

    def createLocalNetworks(self, nns):
        raw_output = []
        for nn_num in range( self.n_local_pred):
            conv1 = self.conv1[nn_num](nns[nn_num])
            bn1 = self.batchnorm[nn_num][0](conv1)
            relu1 = F.relu(bn1)

            conv2 = self.conv2[nn_num](relu1)
            bn2 = self.batchnorm[nn_num][1](conv2)
            relu2 = F.relu(bn2)
            
            conv3 = self.conv3[nn_num](relu2)
            bn3 = self.batchnorm[nn_num][2](conv3)
            relu3 = F.relu(bn3)
            
            relu3 = relu3.view(self.batch_size, -1)
            #print (relu3.size(), self.len_sizes[i][-1] * self.channels[-1]) 
            #print (self.len_sizes[0], len(self.len_sizes))
            
            fc1 = self.fc1[nn_num](relu3)
            relu4 = F.relu(fc1)
            do1 = self.dropout(relu4)
            
            fc2 = self.fc2[nn_num](do1)
            relu5 = F.relu(fc2)            

            do2 = self.dropout(relu5)

            fc3 = self.fc3[nn_num](do2)
            
            raw_output.append(F.log_softmax(fc3, dim = 1))
        return raw_output
    
    def _loss(self, raw_output, targets):
        
        if self.use_cuda:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_()).cuda()
        else:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_())
            
    
        for nn_num in range( self.n_local_pred):
            scores = scores + raw_output[nn_num]
        scores = scores/self.n_local_pred
        
        if self.use_cuda:
            targets = Variable(torch.LongTensor(targets)).cuda()
        else:
            targets = Variable(torch.LongTensor(targets))
            
        loss = F.cross_entropy(scores, targets)
        return loss
    
    def accurancy(self, input_batch, targets):
        self.batch_size = input_batch.shape[0]
        nns  = self.preprocessing(input_batch)
        raw_output = self.createLocalNetworks(nns)
        
        if self.use_cuda:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_()).cuda()
        else:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_())
        
    
        for nn_num in range( self.n_local_pred):
            scores = scores + raw_output[nn_num]
        scores = scores/self.n_local_pred
            
        _, index = torch.max(scores,1)
        index = index.cpu().data.numpy()
        acc = (index == targets).mean()
        return acc
    
    def forward(self, input_batch, targets):
        self.batch_size = input_batch.shape[0]
        nns  = self.preprocessing(input_batch)
        raw_output = self.createLocalNetworks(nns)
        loss = self._loss(raw_output, targets)
        return loss

In [16]:
logdir_root = './logdir'
model = LikeDislikeModel(31, 3, L_sizes = [500],
                 net = 3,
                 fc_sizes = [7500, 1500],
                 n_local_pred = 1,
                 f_sizes = [3, 3, 5], 
                 channels = [256, 128, 64], 
                 strides = [2, 2, 3], 
                 use_cuda = True)

if model.use_cuda:
    model.cuda()
#model.load_state_dict(torch.load(logdir_root +  '/block0-loss=0.499_model.txt'))
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay = 0.03)

In [148]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.000009, weight_decay = 0.12)

In [17]:
logdir_root = './logdir2'
#ValLossFile = open(logdir_root+'/val_loss.txt', 'w')

iterations = 43
epochs = 1000

log_loss = np.zeros(epochs*iterations)
block = 0

for epoch in range(epochs):
    tr_data, tr_targs = get_data(data_set, targets, 256)
    indexes = np.arange(iterations)
    np.random.shuffle(indexes)
    tr_data = tr_data[indexes]
    tr_targs = tr_targs[indexes]
    for iteration in range(iterations):
        batch = tr_data[iteration]
        b_targets = tr_targs[iteration]
        b_indexes = np.arange(256)
        np.random.shuffle(b_indexes)
        batch = batch[b_indexes]
        b_targets = b_targets[b_indexes]
        model.zero_grad()
        loss = model(batch, b_targets)
        log_loss[epoch*iterations + iteration] = loss.data
        loss.backward()
        optimizer.step()
    
    #if epoch % 10 == 0:
    
    if model.use_cuda:
        v_acc = model.accurancy(val_data, val_targs)
        acc =  model.accurancy(batch, b_targets)
        v_loss = model(val_data, val_targs)
        print('epoch {:d}, loss = {:.3f}, val_loss = {:.3f}, val_acc = {:.3f}, train_acc = {:.3f}'
              .format(epoch, loss.cpu().data[0], v_loss.cpu().data[0], v_acc, acc))
        ModelFile = open(logdir_root+'/block{:d}-loss={:.3f}_model.txt'.format(block, loss.cpu().data[0]), 'w')
        #ValLossFile.write('{:.3f} {:.3f}\n'.format(acc, v_acc))
    else:
        print('block {:d}, loss = {:.3f}'
              .format(block, loss.data[0]))
        ModelFile = open(logdir_root+'/block{:d}-loss={:.3f}_model.txt'.format(block, loss.data[0]), 'w')
        #ValLossFile.write('{:.3f} {:.3f}\n'.format(loss.data[0], v_loss.data[0]))
   # block += 1

    torch.save(model.state_dict(), ModelFile)
    ModelFile.close()
#np.savetxt('log_loss.txt', log_loss, fmt='%1.3f')

epoch 0, loss = 1.076, val_loss = 1.082, val_acc = 0.391, train_acc = 0.449
epoch 1, loss = 1.033, val_loss = 1.069, val_acc = 0.404, train_acc = 0.465
epoch 2, loss = 0.994, val_loss = 1.031, val_acc = 0.396, train_acc = 0.477
epoch 3, loss = 0.855, val_loss = 0.850, val_acc = 0.680, train_acc = 0.598
epoch 4, loss = 0.674, val_loss = 0.639, val_acc = 0.669, train_acc = 0.629
epoch 5, loss = 0.660, val_loss = 0.594, val_acc = 0.677, train_acc = 0.594
epoch 6, loss = 0.637, val_loss = 0.575, val_acc = 0.685, train_acc = 0.613
epoch 7, loss = 0.594, val_loss = 0.574, val_acc = 0.682, train_acc = 0.660
epoch 8, loss = 0.595, val_loss = 0.558, val_acc = 0.688, train_acc = 0.629
epoch 9, loss = 0.593, val_loss = 0.562, val_acc = 0.685, train_acc = 0.652
epoch 10, loss = 0.570, val_loss = 0.556, val_acc = 0.677, train_acc = 0.652


KeyboardInterrupt: 

In [13]:
np.savetxt('log_loss.txt', log_loss, fmt='%1.3f')

In [24]:
tst_data.shape

(179, 62, 1200)

In [70]:
test = np.zeros((256, 62, 1200))
for i in range (tst_data.shape[0]):
    test[i] = tst_data[i]

In [44]:
nns = model.preprocessing(batch)
raw_output = model.createLocalNetworks(nns)
scores = Variable(torch.FloatTensor(model.batch_size, model.out_size).zero_()).cuda()
for nn_num in range( model.n_local_pred):
    scores = scores + raw_output[nn_num]

In [45]:
#scores = scores[:tst_data.shape[0]]

print (scores.shape)
_, index = torch.max(scores,1)
index = index.cpu().data.numpy()
print ((index == b_targets).mean())

torch.Size([256, 3])
0.53125


In [13]:
print (model.accurancy(tst_data, tst_targs))

0.773333333333
